# 004 PCA & Clustering With Python

* Written by Alexandre Gazagnes
* Last update: 2024-02-01

## About 

Context : 

You're an export project manager for a major food manufacturer. You are in charge of poultry departement.
You have been asked to identify segments of countries within the company's database in order to target them with personalized marketing campaigns.

Data  : 

After a quick look on the internet, you find a very interesting dataset on the FAO website. It contains a list of countries with various indicators. You decide to use this dataset to identify segments of countries.

You can download the "raw" dataset [here](https://www.fao.org/faostat/fr/#data/QCL).

**You can also use a preprocessed version of the dataset [here](https://gist.githubusercontent.com/AlexandreGazagnes/7cc1176dfe0e281c45e0119210187ae2/raw/4e5c4caac35b88fcedea810f470957b7c5f9b5af/chicken_cleaned.csv).**

Mission :

Your objective is to

- Take a quick tour of the data to understand the data set 

- Clean up the dataset if necessary 

- Perform clustering with Kmeans and Agglomerative Clustering, focusing on countries with large potential markets: populous countries, wealthy countries and/or countries with high import levels

- You need to be able to understand and explain the clusters you've created.

## Preliminaries

### System

These commands will display the system information:

Uncomment theses lines if needed. 

In [1]:
# pwd

In [2]:
# cd ..

In [3]:
# ls

In [4]:
# cd ..

In [5]:
# ls

These commands will install the required packages:

In [6]:
# !pip install pandas matplotlib seaborn plotly scikit-learn

This command will download the dataset:

In [7]:
!wget https://gist.githubusercontent.com/AlexandreGazagnes/7cc1176dfe0e281c45e0119210187ae2/raw/4e5c4caac35b88fcedea810f470957b7c5f9b5af/chicken_cleaned.csv

--2024-04-02 12:39:25--  https://gist.githubusercontent.com/AlexandreGazagnes/7cc1176dfe0e281c45e0119210187ae2/raw/4e5c4caac35b88fcedea810f470957b7c5f9b5af/chicken_cleaned.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6409 (6,3K) [text/plain]
Saving to: ‘chicken_cleaned.csv.1’

chicken_cleaned.csv 100%[===================>]   6,26K  --.-KB/s    in 0,01s   

2024-04-02 12:39:26 (468 KB/s) - ‘chicken_cleaned.csv.1’ saved [6409/6409]



### Import 

Import data libraries:

In [8]:
import pandas as pd
import numpy as np

Import Graphical libraries:

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Import Machine Learning libraries:

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.datasets import load_iris
from scipy.cluster.hierarchy import dendrogram, linkage

### Get the data

1st option : Download the dataset from the web

In [11]:
url = "https://gist.githubusercontent.com/AlexandreGazagnes/7cc1176dfe0e281c45e0119210187ae2/raw/4e5c4caac35b88fcedea810f470957b7c5f9b5af/chicken_cleaned.csv"
df = pd.read_csv(url)
df.head()

,zone,dispo_int,import,dispo_alim,population,dependence,delta
0,Afghanistan,57.0,29.0,5.0,36296000,0.508772,29.0
1,Algérie,277.0,2.0,22.0,41389000,0.007220,2.0
2,Angola,319.0,277.0,35.0,29816000,0.868339,277.0
3,Argentine,1962.0,8.0,182.0,43937000,0.004077,-199.0
4,Australie,1171.0,16.0,192.0,24584000,0.013664,-26.0


2nd Option : Read data from a file

In [12]:
# or

# fn = "./chicken_cleaned.csv"
# df = pd.read_csv(fn)
# df.head()

3rd Option : Load a toy dataset

In [13]:
# or

# data = load_iris()
# df = pd.DataFrame(data.data, columns=data.feature_names)
# df["Species"] = data.target
# df.head()

## Quick Data Viz

Write a function to display the correlation matrix:

In [ ]:
# With a function

def make_corr_heatmap(df):
    corr = df.select_dtypes(include="number").corr()
    mask = np.triu(corr)
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", vmin=-1, vmax=1, mask=mask)

Make a pairplot of the numerical columns:

This visualization can be slow with large datasets. 
Use VIZ = True / False to enable / disable the visualization.

In [ ]:
VIZ = False # Enable this with True
if VIZ : 
    # write your code

## Principal Component Analysis

### Initialisation and fit

Initialize a PCA : 

Fit : 

Here is our new dataset : 

Use pandas to create a DataFrame : 

### Analyse the components

Recompute the first value : 

In [ ]:
(-0.37 * 0.66) + (-0.44 * 0.11) + (-1.1 * 0.34) + (-0.15 * 0.46) + (-0.46 * -0.1) + (0.11*-0.46)

### Plot explained variance

The explained variance ratio is pre-computed : 

We can plot it : 

A better feature is the cumulative variance : 

We can plot it : 

### Correlation graph

In [ ]:
def correlation_graph(
    X_scaled,
    pca,
    dim: list = [0, 1],
):
    """Affiche le graphe des correlations

    Positional arguments :
        X_scaled : DataFrame | np.array : le dataset scaled
        pca : PCA : l'objet PCA déjà fitté

    Optional arguments :
        dim : list ou tuple : le couple x,y des plans à afficher, exemple [0,1] pour F1, F2
    """

    # Extrait x et y
    x, y = dim

    # features
    features = X_scaled.columns

    # Taille de l'image (en inches)
    fig, ax = plt.subplots(figsize=(10, 9))

    # Pour chaque composante :
    for i in range(0, pca.components_.shape[1]):
        # Les flèches
        ax.arrow(
            0,
            0,
            pca.components_[x, i],
            pca.components_[y, i],
            head_width=0.07,
            head_length=0.07,
            width=0.02,
        )

        # Les labels
        plt.text(
            pca.components_[x, i] + 0.05,
            pca.components_[y, i] + 0.05,
            features[i],
        )

    # Affichage des lignes horizontales et verticales
    plt.plot([-1, 1], [0, 0], color="grey", ls="--")
    plt.plot([0, 0], [-1, 1], color="grey", ls="--")

    # Nom des axes, avec le pourcentage d'inertie expliqué
    plt.xlabel(
        "F{} ({}%)".format(
            x + 1, round(100 * pca.explained_variance_ratio_[x], 1)
        )
    )
    plt.ylabel(
        "F{} ({}%)".format(
            y + 1, round(100 * pca.explained_variance_ratio_[y], 1)
        )
    )

    # title     
    plt.title("Cercle des corrélations (F{} et F{})".format(x + 1, y + 1))

    # Le cercle
    an = np.linspace(0, 2 * np.pi, 100)
    plt.plot(np.cos(an), np.sin(an))  # Add a unit circle for scale

    # Axes et display
    plt.axis("equal")
    plt.show(block=False)

In [ ]:
correlation_graph(# Add arguments)

In [ ]:
correlation_graph(# Add arguments)

In [ ]:
correlation_graph(# Add arguments)

### Factorial planes

In [ ]:
def factorial_planes(
    X_,
    pca,
    dim,
    labels: list = None,
    clusters: list = None,
    figsize: list = [12, 10],
    fontsize = 14,
):

    """Affiche les plans factoriels
    """

    x, y = dim

    dtypes = (pd.DataFrame, np.ndarray, pd.Series, list, tuple, set)
    if not isinstance(labels, dtypes):
        labels = []
    if not isinstance(clusters, dtypes):
        clusters = []

    # Initialisation de la figure
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    if len(clusters):
        sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y], hue=clusters)
    else:
        sns.scatterplot(data=None, x=X_[:, x], y=X_[:, y])

    # Si la variable pca a été fournie, on peut calculer le % de variance de chaque axe
    v1 = str(round(100 * pca.explained_variance_ratio_[x])) + " %"
    v2 = str(round(100 * pca.explained_variance_ratio_[y])) + " %"

    # Nom des axes, avec le pourcentage d'inertie expliqué
    ax.set_xlabel(f"F{x+1} {v1}")
    ax.set_ylabel(f"F{y+1} {v2}")

    # Valeur x max et y max
    x_max = np.abs(X_[:, x]).max() * 1.1
    y_max = np.abs(X_[:, y]).max() * 1.1

    # On borne x et y
    ax.set_xlim(left=-x_max, right=x_max)
    ax.set_ylim(bottom=-y_max, top=y_max)

    # Affichage des lignes horizontales et verticales
    plt.plot([-x_max, x_max], [0, 0], color="grey", alpha=0.8)
    plt.plot([0, 0], [-y_max, y_max], color="grey", alpha=0.8)

    # Affichage des labels des points
    if len(labels):
        for i, (_x, _y) in enumerate(X_[:, [x, y]]):
            plt.text(
                _x, _y + 0.05, labels[i], fontsize=fontsize, ha="center", va="center"
            )

    # Titre et display
    plt.title(f"Projection des individus (sur F{x+1} et F{y+1})")
    plt.show()

In [ ]:
factorial_planes(# add arguments)

In [ ]:
factorial_planes(# add arguments)

## Kmeans

### First Naive Clustering

Initialize the Kmeans model with arbitrary number of clusters:

Fit the Kmeans model to the data:

Use alphanumerical labels for the clusters:

In [ ]:
labels_values = {0:"A", 1:"B", 2:"C", 3:"D", 4:"E", 5:"F", 6:"G", 7:"H", 8:"I", 9:"J"}



Create a copy of the original dataset and add labels:

What about A cluster?

What about B cluster?

Etc etc...

Build a boxplot for each cluster:

Same but with plotly:

### Find the best number of clusters

Compute the inertia for different number of clusters:

In [ ]:
# WCSS : Within-Cluster-Sum-of-Squares



Plot the inertia for different number of clusters:

Same but with davies bouldin score:

In [ ]:
# DB Score 



Same but with silhouette score:

In [ ]:
# silhouette Score



### Use the best number of clusters

Fit the Kmeans model to the data with the best number of clusters:

Have a look to labels

Use string values for labels:

Update _df dataset with labels:

Have a look to A cluster:

Have a look to B cluster:

Etc etc...

Use boxplot to visualize the clusters:

The D cluster is interesting:

## OPTIONAL Agglomerative Clustering (Hierarchical Clustering)

Create a model with the best number of clusters:

Fit the model to the data:

Have a look to labels : 

Use string values for labels:

Update _df dataset with new labels:

Have a look to A cluster:

Have a look to B cluster:

Etc etc...

Use Plotly to visualize the clusters:

Plot the dendrogram

In [ ]:

plt.figure(figsize=(10, 5))
plt.xlabel("sample index")
plt.ylabel("distance")
z = linkage(X_scaled, method="ward")
dendrogram(
    z,
    leaf_rotation=90,
    p=5,
    color_threshold=10,
    leaf_font_size=10,
    truncate_mode="level",
)
plt.tight_layout()